# 0.回顾numpy的级联
- 1.生成2各3*3的矩阵，对其分别进行两个维度上的级联

In [1]:
import numpy as np
import pandas as pd

In [2]:
nd = np.random.randint(0,10,size=(3,3))
nd

array([[1, 1, 8],
       [1, 3, 5],
       [9, 2, 8]])

In [3]:
nd1 = np.random.randint(0,10,size=(3,3))
nd1

array([[8, 4, 3],
       [0, 6, 2],
       [6, 5, 2]])

In [4]:
#0是第一维的方向：行
np.concatenate((nd,nd1),axis=0)

array([[1, 1, 8],
       [1, 3, 5],
       [9, 2, 8],
       [8, 4, 3],
       [0, 6, 2],
       [6, 5, 2]])

In [5]:
np.concatenate((nd,nd1),axis=1)

array([[1, 1, 8, 8, 4, 3],
       [1, 3, 5, 0, 6, 2],
       [9, 2, 8, 6, 5, 2]])

- 定义生成一个DataFrame的函数：

In [17]:
def make_df(cols,inds):
    data = {c:[c+str(i) for i in inds] for c in cols}
    print(data)
    return pd.DataFrame(data,index = inds,columns = cols)

In [18]:
make_df(['A','B'],[1,2])

{'A': ['A1', 'A2'], 'B': ['B1', 'B2']}


,A,B
1,A1,B1
2,A2,B2


# 1.使用pd.concat()级联
- Pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```python
Signature: pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
```
## 1.1简单级联
- 和np.concatenate一样，优先增加行数(默认axis=0)

In [8]:
df1 = make_df(list('AB'),[0,1])
df2 = make_df(list('AB'),[2,3])
pd.concat([df1,df2])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [9]:
pd.concat((df1,df2))

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [10]:
#可以通过设置axis来改变级联方向
pd.concat((df1,df2),axis=1)

,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3


In [11]:
# 注意，index在级联时是可以重复的
df3 = make_df(list('AB'),[0,1,2])
df4 = make_df(list('AB'),[1,2,3])

In [12]:
df5 = pd.concat((df3,df4))
df5

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
1,A1,B1
2,A2,B2
3,A3,B3


In [13]:
df5.loc[[1,2]]

,A,B
1,A1,B1
1,A1,B1
2,A2,B2
2,A2,B2


In [14]:
#可以选择忽略ignore_index,重新索引
pd.concat((df3,df4),ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A1,B1
4,A2,B2
5,A3,B3


In [15]:
#或者使用多层索引keys
x = make_df(list('XY'),['a','b'])
y = make_df(list('XY'),['A','B'])
pd.concat((x,y))

,X,Y
a,Xa,Ya
b,Xb,Yb
A,XA,YA
B,XB,YB


In [16]:
#keys可以增加索引，使得合并后的数据更清晰
pd.concat((x,y),keys=['x','y'])

X   Y
x a  Xa  Ya
  b  Xb  Yb
y A  XA  YA
  B  XB  YB

## 1.2不匹配级联
- 不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时航索引不一致
- 有3种连接方式：
- 外连接：补NaN（默认模式）

In [19]:
df1 = make_df(['A','B'],[1,3])
df2 = make_df(['B','C'],[2,4])

{'A': ['A1', 'A3'], 'B': ['B1', 'B3']}
{'B': ['B2', 'B4'], 'C': ['C2', 'C4']}


In [23]:
pd.concat((df1,df2),join='outer')

F:\soft\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,A,B,C
1,A1,B1,NaN
3,A3,B3,NaN
2,NaN,B2,C2
4,NaN,B4,C4


- 内连接：只连接匹配的项

In [22]:
pd.concat((df1,df2),join='inner')

,B
1,B1
3,B3
2,B2
4,B4


- 连接指定轴join_axes

In [24]:
df3 = make_df(list('ACD'),[0,1,2])
df4 = make_df(list('CDF'),[3,4,5])

{'A': ['A0', 'A1', 'A2'], 'C': ['C0', 'C1', 'C2'], 'D': ['D0', 'D1', 'D2']}
{'C': ['C3', 'C4', 'C5'], 'D': ['D3', 'D4', 'D5'], 'F': ['F3', 'F4', 'F5']}


In [25]:
pd.concat((df3,df4))

F:\soft\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,A,C,D,F
0,A0,C0,D0,NaN
1,A1,C1,D1,NaN
2,A2,C2,D2,NaN
3,NaN,C3,D3,F3
4,NaN,C4,D4,F4
5,NaN,C5,D5,F5


In [27]:
#join_axes 以某一个DataFrame列索引为新的索引值
pd.concat((df3,df4),join_axes=[df4.columns])

,C,D,F
0,C0,D0,NaN
1,C1,D1,NaN
2,C2,D2,NaN
3,C3,D3,F3
4,C4,D4,F4
5,C5,D5,F5


## 1.3使用append()函数级联
- 由于级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [28]:
df1 = make_df(['大众','雷克萨斯'],[0,1,2,3,4])
df2 = make_df(['大众','雷克萨斯'],[5,6,7,8,9])

{'大众': ['大众0', '大众1', '大众2', '大众3', '大众4'], '雷克萨斯': ['雷克萨斯0', '雷克萨斯1', '雷克萨斯2', '雷克萨斯3', '雷克萨斯4']}
{'大众': ['大众5', '大众6', '大众7', '大众8', '大众9'], '雷克萨斯': ['雷克萨斯5', '雷克萨斯6', '雷克萨斯7', '雷克萨斯8', '雷克萨斯9']}


In [29]:
df1.append(df2)

,大众,雷克萨斯
0,大众0,雷克萨斯0
1,大众1,雷克萨斯1
2,大众2,雷克萨斯2
3,大众3,雷克萨斯3
4,大众4,雷克萨斯4
5,大众5,雷克萨斯5
6,大众6,雷克萨斯6
7,大众7,雷克萨斯7
8,大众8,雷克萨斯8
9,大众9,雷克萨斯9


# 2.使用pd.merge()合并
- merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并
- 使用pd.merge()合并时，除自动根除两者相同columns名称的那一列，作为key来进行合并
- 注意每一列元素的吮吸不要求一致

## 2.1一对一合并

In [30]:
df1 = pd.DataFrame({'employee':['Po','Sara','Danis'],'group':['sail','counting','marcketing']})
df2 = pd.DataFrame({'employee':['Po','Sara','Danis'],'work_time':[2,3,1]})
display(df1,df2)

,employee,group
0,Po,sail
1,Sara,counting
2,Danis,marcketing


,employee,work_time
0,Po,2
1,Sara,3
2,Danis,1


In [32]:
pd.merge(df1,df2)

,employee,group,work_time
0,Po,sail,2
1,Sara,counting,3
2,Danis,marcketing,1


In [33]:
df3 = pd.DataFrame({'employee':['Po','Sara','Bush'],'work_time':[2,3,1]})
pd.merge(df1,df3)

,employee,group,work_time
0,Po,sail,2
1,Sara,counting,3


In [35]:
pd.concat((df1,df2),axis=1)

,employee,group,employee,work_time
0,Po,sail,Po,2
1,Sara,counting,Sara,3
2,Danis,marcketing,Danis,1


## 2.2多对一合并

In [36]:
df1 = pd.DataFrame({'employee':['Po','Sara','Danis'],'group':['sail','counting','marcketing']})
df2 = pd.DataFrame({'employee':['Po','Po','Danis'],'work_time':[2,3,1]})
display(df1,df2)

,employee,group
0,Po,sail
1,Sara,counting
2,Danis,marcketing


,employee,work_time
0,Po,2
1,Po,3
2,Danis,1


In [37]:
pd.merge(df1,df2)

,employee,group,work_time
0,Po,sail,2
1,Po,sail,3
2,Danis,marcketing,1


## 2.3多对多合并

In [38]:
df1 = pd.DataFrame({'employee':['Po','Po','Danis'],'group':['sail','counting','marcketing']})
df2 = pd.DataFrame({'employee':['Po','Po','Danis'],'work_time':[2,3,1]})
display(df1,df2)

,employee,group
0,Po,sail
1,Po,counting
2,Danis,marcketing


,employee,work_time
0,Po,2
1,Po,3
2,Danis,1


In [39]:
#在进行多对多合并时，每一个数据都没有放过
pd.merge(df1,df2)

,employee,group,work_time
0,Po,sail,2
1,Po,sail,3
2,Po,counting,2
3,Po,counting,3
4,Danis,marcketing,1


## 2.4key的规范化
- 使用on=显示指定哪一列位key,当有多个key相同时使用

In [44]:
df3 = pd.DataFrame({'employee':['Po','Summer','Flower'],
                    'group':['sail','marketing','serch'],
                   'salary':[12000,10000,8000]})
df4 = pd.DataFrame({'employee':['Po','Winter','Flower'],
                    'group':['marketing','marketing','serch'],
                   'work_time':[2,1,5]})
display(df3,df4)

,employee,group,salary
0,Po,sail,12000
1,Summer,marketing,10000
2,Flower,serch,8000


,employee,group,work_time
0,Po,marketing,2
1,Winter,marketing,1
2,Flower,serch,5


In [45]:
pd.merge(df3,df4)

,employee,group,salary,work_time
0,Flower,serch,8000,5


In [47]:
pd.merge(df3,df4,on='employee')

,employee,group_x,salary,group_y,work_time
0,Po,sail,12000,marketing,2
1,Flower,serch,8000,serch,5


In [49]:
pd.merge(df3,df4,on='group',suffixes=['_A','_B'])

,employee_A,group,salary,employee_B,work_time
0,Summer,marketing,10000,Po,2
1,Summer,marketing,10000,Winter,1
2,Flower,serch,8000,Flower,5


- 使用left_on和right_on指定左右两边的列作为key，当左右两个的key都不相等时使用

In [54]:
df5 = pd.DataFrame({'employer':['Po','Summer','Flower'],
                    'Team':['sail','marketing','serch'],
                   'salary':[12000,10000,8000]})
df6 = pd.DataFrame({'employee':['Po','Winter','Flower'],
                    'group':['marketing','marketing','serch'],
                   'work_time':[2,1,5]})
display(df5,df6)

,employer,Team,salary
0,Po,sail,12000
1,Summer,marketing,10000
2,Flower,serch,8000


,employee,group,work_time
0,Po,marketing,2
1,Winter,marketing,1
2,Flower,serch,5


In [55]:
pd.merge(df5,df6,left_on='employer',right_on='employee')

,employer,Team,salary,employee,group,work_time
0,Po,sail,12000,Po,marketing,2
1,Flower,serch,8000,Flower,serch,5


In [57]:
pd.merge(df5,df6,left_on='Team',right_on='group')

,employer,Team,salary,employee,group,work_time
0,Summer,marketing,10000,Po,marketing,2
1,Summer,marketing,10000,Winter,marketing,1
2,Flower,serch,8000,Flower,serch,5


## 2.5内合并与外合并
- 内合并：只保留两者都有的key（默认模式）

In [58]:
df1 = pd.DataFrame({'age':[18,19,22,25],'height':[170,165,180,170]})
df2 = pd.DataFrame({'age':[18,20,22,30],'weight':[55,50,60,65]})
display(df1,df2)

,age,height
0,18,170
1,19,165
2,22,180
3,25,170


,age,weight
0,18,55
1,20,50
2,22,60
3,30,65


In [59]:
pd.merge(df1,df2)

,age,height,weight
0,18,170,55
1,22,180,60


- 外合并how='outer'：补NaN

In [60]:
pd.merge(df1,df2,how='outer')

,age,height,weight
0,18,170.0,55.0
1,19,165.0,NaN
2,22,180.0,60.0
3,25,170.0,NaN
4,20,NaN,50.0
5,30,NaN,65.0


- 左合并、右合并：how='left',how='right'

In [61]:
pd.merge(df1,df2,how='left')

,age,height,weight
0,18,170,55.0
1,19,165,NaN
2,22,180,60.0
3,25,170,NaN


In [62]:
pd.merge(df1,df2,how='right')

,age,height,weight
0,18,170.0,55
1,22,180.0,60
2,20,NaN,50
3,30,NaN,65
